In [ ]:
EXERCISE:#Creating a network with just two layers - an input and an output layer, no hidden layer - with 784 and 10 neurons, respectively.
#Train the network using stochastic gradient descent. What classification accuracy can you achieve?

# Libraries
import numpy as np
import random
import pickle
import gzip
# Load the MNIST dataset locally
from tensorflow.keras.datasets import mnist

In [ ]:
"""
mnist_loader
~~~~~~~~~~~~

A library to load the MNIST image data.  For details of the data
structures that are returned, see the doc strings for ``load_data``
and ``load_data_wrapper``.  In practice, ``load_data_wrapper`` is the
function usually called by our neural network code.
"""

#### Libraries
# Standard library
import pickle
import gzip

# Third-party libraries
import numpy as np

def load_data():
    """Return the MNIST data as a tuple containing the training data,
    the validation data, and the test data.

    The ``training_data`` is returned as a tuple with two entries.
    The first entry contains the actual training images.  This is a
    numpy ndarray with 50,000 entries.  Each entry is, in turn, a
    numpy ndarray with 784 values, representing the 28 * 28 = 784
    pixels in a single MNIST image.

    The second entry in the ``training_data`` tuple is a numpy ndarray
    containing 50,000 entries.  Those entries are just the digit
    values (0...9) for the corresponding images contained in the first
    entry of the tuple.

    The ``validation_data`` and ``test_data`` are similar, except
    each contains only 10,000 images.

    This is a nice data format, but for use in neural networks it's
    helpful to modify the format of the ``training_data`` a little.
    That's done in the wrapper function ``load_data_wrapper()``, see
    below.
    """
    f = gzip.open('../data/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f)
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    """Return a tuple containing ``(training_data, validation_data,
    test_data)``. Based on ``load_data``, but the format is more
    convenient for use in our implementation of neural networks.

    In particular, ``training_data`` is a list containing 50,000
    2-tuples ``(x, y)``.  ``x`` is a 784-dimensional numpy.ndarray
    containing the input image.  ``y`` is a 10-dimensional
    numpy.ndarray representing the unit vector corresponding to the
    correct digit for ``x``.

    ``validation_data`` and ``test_data`` are lists containing 10,000
    2-tuples ``(x, y)``.  In each case, ``x`` is a 784-dimensional
    numpy.ndarry containing the input image, and ``y`` is the
    corresponding classification, i.e., the digit values (integers)
    corresponding to ``x``.

    Obviously, this means we're using slightly different formats for
    the training data and the validation / test data.  These formats
    turn out to be the most convenient for use in our neural network
    code."""
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    """Return a 10-dimensional unit vector with a 1.0 in the jth
    position and zeroes elsewhere.  This is used to convert a digit
    (0...9) into a corresponding desired output from the neural
    network."""
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e




In [ ]:
import numpy as np
import random

class Network:
    def __init__(self, sizes):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x) / np.sqrt(x) for x, y in zip(sizes[:-1], sizes[1:])]



    def SGD(self, training_data, epochs, mini_batch_size, eta, test_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent."""
        if test_data:
            n_test = len(test_data)
        n = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size] for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print("Epoch {0}: {1} / {2}".format(j, self.evaluate(test_data), n_test))
            else:
                print("Epoch {0} complete".format(j))

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        mini_batch_size = len(mini_batch)

        # Separate inputs and targets
        mini_batch_inputs, mini_batch_targets = zip(*mini_batch)

        # Convert inputs and targets to matrices
        mini_batch_inputs = np.hstack(mini_batch_inputs).reshape(-1, mini_batch_size)
        mini_batch_targets = np.hstack(mini_batch_targets).reshape(-1, mini_batch_size)

        delta_nabla_b, delta_nabla_w = self.backprop(mini_batch_inputs, mini_batch_targets)

        # Update weights and biases
        self.weights = [w - (eta / mini_batch_size) * nw for w, nw in zip(self.weights, delta_nabla_w)]
        self.biases = [b - (eta / mini_batch_size) * nb for b, nb in zip(self.biases, delta_nabla_b)]


    def backprop(self, x, y):
        """Return a tuple representing the gradient for the cost function C_x."""
        # Initialize gradients
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # Feedforward
        activation = x
        activations = [x]  # List to store all the activations, layer by layer
        zs = []  # List to store all the z vectors, layer by layer

        # Feedforward
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation) + b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)

        # Backward pass
        delta = self.cost_derivative(activations[-1], y) * sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())

        # Update gradients for earlier layers
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l + 1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l - 1].transpose())

        return nabla_b, nabla_w

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural network
        outputs the correct result."""
        test_results = [(np.argmax(self.feedforward(x)), np.argmax(y)) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a) + b)
        return a

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return output_activations - y

def sigmoid(z):
    """The sigmoid function."""
    return 1.0 / (1.0 + np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z) * (1 - sigmoid(z))


In [ ]:
net = Network([784, 10])  # No hidden


In [ ]:
# Train the network using stochastic gradient descent
net.SGD(training_data, epochs=30, mini_batch_size=10, eta=3.0, test_data=test_data)

Epoch 0: 0 / 10000
Epoch 1: 0 / 10000
Epoch 2: 0 / 10000
Epoch 3: 0 / 10000
Epoch 4: 0 / 10000
Epoch 5: 0 / 10000
Epoch 6: 0 / 10000
Epoch 7: 0 / 10000
Epoch 8: 0 / 10000
Epoch 9: 0 / 10000
Epoch 10: 0 / 10000
Epoch 11: 0 / 10000
Epoch 12: 0 / 10000
Epoch 13: 0 / 10000
Epoch 14: 0 / 10000
Epoch 15: 0 / 10000
Epoch 16: 0 / 10000
Epoch 17: 0 / 10000
Epoch 18: 0 / 10000
Epoch 19: 0 / 10000
Epoch 20: 0 / 10000
Epoch 21: 0 / 10000
Epoch 22: 0 / 10000
Epoch 23: 0 / 10000
Epoch 24: 0 / 10000
Epoch 25: 0 / 10000
Epoch 26: 0 / 10000
Epoch 27: 0 / 10000
Epoch 28: 0 / 10000
Epoch 29: 0 / 10000


In [ ]:
net =Network([784, 10])
net.SGD(training_data, 30, 10, 100.0, test_data=test_data)

Epoch 0: 1032 / 10000
Epoch 1: 1032 / 10000
Epoch 2: 1032 / 10000
Epoch 3: 1032 / 10000
Epoch 4: 1032 / 10000
Epoch 5: 1032 / 10000
Epoch 6: 1032 / 10000
Epoch 7: 1032 / 10000
Epoch 8: 1032 / 10000
Epoch 9: 1032 / 10000
Epoch 10: 1032 / 10000
Epoch 11: 1032 / 10000
Epoch 12: 1032 / 10000
Epoch 13: 1032 / 10000
Epoch 14: 1032 / 10000
Epoch 15: 1032 / 10000
Epoch 16: 1032 / 10000
Epoch 17: 1032 / 10000
Epoch 18: 1032 / 10000
Epoch 19: 1032 / 10000
Epoch 20: 1032 / 10000
Epoch 21: 1032 / 10000
Epoch 22: 1032 / 10000
Epoch 23: 1032 / 10000
Epoch 24: 1032 / 10000
Epoch 25: 1032 / 10000
Epoch 26: 1032 / 10000
Epoch 27: 1032 / 10000
Epoch 28: 1032 / 10000
Epoch 29: 1032 / 10000


In [ ]:
#EXERCISE ONE (PART ONE )


import numpy as np

def perceptron_output(x, w, b):
    """Output of a perceptron."""
    return 1 if np.dot(w, x) + b > 0 else 0

def multiply_weights_biases(w, b, c):
    """Multiply weights and biases by a positive constant c."""
    w_c = c * w
    b_c = c * b
    return w_c, b_c

# Example: weights, biases, and input
original_w = np.array([2, -1])  # Example weights
original_b = 1  # Example bias
input_vector = np.array([1, 2])  # Example input

# Calculate original output
original_output = perceptron_output(input_vector, original_w, original_b)

# Multiply weights and biases by a positive constant c
c_value = 2  # You can change this value
new_w, new_b = multiply_weights_biases(original_w, original_b, c_value)

# Calculate modified output
modified_output = perceptron_output(input_vector, new_w, new_b)

# Print results
print(f"Original Output: {original_output}")
print(f"Modified Output (after multiplying by c={c_value}): {modified_output}")

Original Output: 1
Modified Output (after multiplying by c=2): 1


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def perceptron_output(x, w, b):
    """Output of a perceptron."""
    return 1 if np.dot(w, x) + b > 0 else 0

def sigmoid_neuron_output(x, w, b):
    """Output of a sigmoid neuron."""
    z = np.dot(w, x) + b
    return 1 / (1 + np.exp(-z))

def simulate_behavior(c_values, perceptron=True):
    """Simulate behavior with weights and biases multiplied by a constant c."""
    # Define weights, biases, and input
    w = np.array([2, -1])  # Example weights
    b = 1  # Example bias
    x = np.array([1, 2])  # Example input

    outputs_original = []
    outputs_modified = []

    for c in c_values:
        # Multiply weights and bias by c
        w_c = c * w
        b_c = c * b

        # Calculate outputs
        if perceptron:
            output_original = perceptron_output(x, w, b)
            output_modified = perceptron_output(x, w_c, b_c)
        else:
            output_original = sigmoid_neuron_output(x, w, b)
            output_modified = sigmoid_neuron_output(x, w_c, b_c)

        outputs_original.append(output_original)
        outputs_modified.append(output_modified)

    return outputs_original, outputs_modified

# Test with perceptron
c_values = np.linspace(1, 10, 100)
outputs_perceptron = simulate_behavior(c_values, perceptron=True)

# Test with sigmoid neuron
outputs_sigmoid = simulate_behavior(c_values, perceptron=False)
